In [1]:
#typical ML imports (from NN example ipynb)
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from glob import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import *
from tensorflow.keras import Sequential
#from utils import * 

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

plt.rcParams['savefig.dpi'] = 400
plt.rcParams['font.size'] = 13
plt.rcParams["legend.frameon"] = False

2022-12-22 18:13:18.381803: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Open and Check CESM Data

In [ ]:
# Define ens (ensemble) and member (member number) so that all subsequent lines would be valid in a loop for a testbed approach
ens = 'CESM'
member = '009'

# Define date range
# In this case, CESM from 1982-2017 and SOCAT from 1970-2022, so keep CESM full range and will trim SOCAT
date_start = '1982-02'
date_end = '2017-01'
# LATER: check input[each] date and lat/lon and make sure all coming in reasonably

In [ ]:
base_url = '/home/julias/MLEE-final-project/raw_data/{}_2D_mon_{}{}_1x1_198201-201701.nc'
XCO2_url = '/home/julias/MLEE-final-project/raw_data/{}_1D_mon_{}{}_native_198201-201701.nc'
mask_path = '/home/julias/MLEE-final-project/raw_data/create_mask.nc'

variables = {'Chl','FG-CO2','iceFrac','MLD','pATM','pCO2','SSS','SST','U10','XCO2'}

full_potential_variables_list = []
inputs = {}
inputs['socat_mask'] = xr.open_dataset(mask_path).socat_mask.sel(time=slice(date_start,date_end))
#time = inputs['socat_mask']

for variable in variables:

    if variable=='Chl':    
        Chl_path = base_url.format(variable,ens,member)
        inputs['Chl'] = xr.open_dataset(Chl_path).Chl.sel(time=slice(date_start,date_end)).assign_coords(time=inputs['socat_mask'].time)
        full_potential_variables_list.append(Chl_path)
    if variable=='MLD':    
        MLD_path = base_url.format(variable,ens,member)
        inputs['MLD'] = xr.open_dataset(MLD_path).MLD.sel(time=slice(date_start,date_end)).assign_coords(time=inputs['socat_mask'].time)
        full_potential_variables_list.append(MLD_path)
    if variable=='pATM':    
        pATM_path = base_url.format(variable,ens,member)
        inputs['pATM'] = xr.open_dataset(pATM_path).pATM.sel(time=slice(date_start,date_end)).assign_coords(time=inputs['socat_mask'].time)
        full_potential_variables_list.append(pATM_path)
    if variable=='pCO2':    
        pCO2_path = base_url.format(variable,ens,member)
        inputs['pCO2'] = xr.open_dataset(pCO2_path).pCO2.sel(time=slice(date_start,date_end)).assign_coords(time=inputs['socat_mask'].time)
        full_potential_variables_list.append(pCO2_path)
    if variable=='SSS':    
        SSS_path = base_url.format(variable,ens,member)
        inputs['SSS'] = xr.open_dataset(SSS_path).SSS.sel(time=slice(date_start,date_end)).assign_coords(time=inputs['socat_mask'].time)    
        full_potential_variables_list.append(SSS_path)
    if variable=='SST':    
        SST_path = base_url.format(variable,ens,member)
        inputs['SST'] = xr.open_dataset(SST_path).SST.sel(time=slice(date_start,date_end)).assign_coords(time=inputs['socat_mask'].time)
        full_potential_variables_list.append(SST_path)
        
    # Note that XCO2 requires a slightly different URL
    if variable=='XCO2':
        XCO2_path = XCO2_url.format(variable,ens,member)
        inputs['XCO2'] = xr.open_dataset(XCO2_path).XCO2.sel(time=slice(date_start,date_end)).assign_coords(time=inputs['socat_mask'].time)
        full_potential_variables_list.append(XCO2_path)
        
    if variable=='FG-CO2' or variable=='iceFrac' or variable=='U10':
        full_potential_variables_list.append(base_url.format(variable,ens,member))
    

In [ ]:
for i in inputs:        
    if i != 'XCO2' and i != 'socat_mask':
        inputs[i] = inputs[i].assign_coords(xlon=(((inputs[i].xlon+180)%360)-180)) # wraps from 0 to 360 to -180 to 180
        inputs[i] = inputs[i].sortby(inputs[i].xlon)

In [ ]:
#inputs['XCO2']

In [ ]:
#inputs['SSS']

In [ ]:
#inputs['socat_mask']

In [ ]:
DS = xr.merge([inputs['SSS'], inputs['SST'], inputs['MLD'], inputs['Chl'], inputs['pCO2'], inputs['socat_mask']]) #
    


#    inputs[variable] = xr.open_dataset(base_url.format(variable,ens,member)).variables

In [ ]:
DS

In [ ]:
full_potential_variables_list

In [ ]:
inputs['XCO2']

In [2]:
def import_model_data(ens, member, date_start, date_end):
    
    base_url = '/home/julias/MLEE-final-project/raw_data/{}_2D_mon_{}{}_1x1_198201-201701.nc'
    XCO2_url = '/home/julias/MLEE-final-project/raw_data/{}_1D_mon_{}{}_native_198201-201701.nc'
    mask_path = '/home/julias/MLEE-final-project/raw_data/create_mask.nc'

    variables = {'Chl','FG-CO2','iceFrac','MLD','pATM','pCO2','SSS','SST','U10','XCO2'}

    full_potential_variables_list = []
    inputs = {}
    inputs['socat_mask'] = xr.open_dataset(mask_path).socat_mask.sel(time=slice(date_start,date_end))
#time = inputs['socat_mask']

    for variable in variables:

        if variable=='Chl':    
            Chl_path = base_url.format(variable,ens,member)
            inputs['Chl'] = xr.open_dataset(Chl_path).Chl.sel(time=slice(date_start,date_end)).assign_coords(time=inputs['socat_mask'].time)
            full_potential_variables_list.append(Chl_path)
        if variable=='MLD':    
            MLD_path = base_url.format(variable,ens,member)
            inputs['MLD'] = xr.open_dataset(MLD_path).MLD.sel(time=slice(date_start,date_end)).assign_coords(time=inputs['socat_mask'].time)
            full_potential_variables_list.append(MLD_path)
        if variable=='pATM':    
            pATM_path = base_url.format(variable,ens,member)
            inputs['pATM'] = xr.open_dataset(pATM_path).pATM.sel(time=slice(date_start,date_end)).assign_coords(time=inputs['socat_mask'].time)
            full_potential_variables_list.append(pATM_path)
        if variable=='pCO2':    
            pCO2_path = base_url.format(variable,ens,member)
            inputs['pCO2'] = xr.open_dataset(pCO2_path).pCO2.sel(time=slice(date_start,date_end)).assign_coords(time=inputs['socat_mask'].time)
            full_potential_variables_list.append(pCO2_path)
        if variable=='SSS':    
            SSS_path = base_url.format(variable,ens,member)
            inputs['SSS'] = xr.open_dataset(SSS_path).SSS.sel(time=slice(date_start,date_end)).assign_coords(time=inputs['socat_mask'].time)    
            full_potential_variables_list.append(SSS_path)
        if variable=='SST':    
            SST_path = base_url.format(variable,ens,member)
            inputs['SST'] = xr.open_dataset(SST_path).SST.sel(time=slice(date_start,date_end)).assign_coords(time=inputs['socat_mask'].time)
            full_potential_variables_list.append(SST_path)
        
    # Note that XCO2 requires a slightly different URL
        if variable=='XCO2':
            XCO2_path = XCO2_url.format(variable,ens,member)
            inputs['XCO2'] = xr.open_dataset(XCO2_path).XCO2.sel(time=slice(date_start,date_end)).assign_coords(time=inputs['socat_mask'].time)
            full_potential_variables_list.append(XCO2_path)
        
        if variable=='FG-CO2' or variable=='iceFrac' or variable=='U10':
            full_potential_variables_list.append(base_url.format(variable,ens,member))
    
    for i in inputs:        
        if i != 'XCO2' and i != 'socat_mask':
            inputs[i] = inputs[i].assign_coords(xlon=(((inputs[i].xlon+180)%360)-180)) # wraps from 0 to 360 to -180 to 180
            inputs[i] = inputs[i].sortby(inputs[i].xlon)    
        
    DS = xr.merge([inputs['SSS'], inputs['SST'], inputs['MLD'], inputs['Chl'], inputs['pCO2'], inputs['socat_mask']])
    
    return DS, inputs['XCO2']

In [3]:
# Define ens (ensemble) and member (member number) so that all subsequent lines would be valid in a loop for a testbed approach
ens = 'CESM'
member = '009'

# Define date range
# In this case, CESM from 1982-2017 and SOCAT from 1970-2022, so keep CESM full range and will trim SOCAT
date_start = '1982-02'
date_end = '2017-01'
# LATER: check input[each] date and lat/lon and make sure all coming in reasonably

In [6]:
CESM_data, XCO2_data = import_model_data(ens, member, date_start, date_end)

In [7]:
CESM_data

<xarray.Dataset>
Dimensions:     (xlon: 360, ylat: 180, time: 420)
Coordinates:
  * xlon        (xlon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * ylat        (ylat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * time        (time) datetime64[ns] 1982-02-15 1982-03-15 ... 2017-01-15
Data variables:
    SSS         (time, ylat, xlon) float32 ...
    SST         (time, ylat, xlon) float32 ...
    MLD         (time, ylat, xlon) float32 ...
    Chl         (time, ylat, xlon) float32 ...
    pCO2        (time, ylat, xlon) float64 ...
    socat_mask  (time, ylat, xlon) float64 ...
Attributes:
    long_name:     Sea Surface Salinity
    units:         gram/kilogram
    grid_loc:      3111
    cell_methods:  time: mean z_t: mean

In [8]:
XCO2_data

<xarray.DataArray 'XCO2' (time: 420)>
array([340.84854, 340.96225, 341.07544, ..., 406.5084 , 406.7395 , 406.97128],
      dtype=float32)
Coordinates:
    TLONG    float64 ...
    TLAT     float64 ...
  * time     (time) datetime64[ns] 1982-02-15 1982-03-15 ... 2017-01-15
Attributes:
    long_name:     Atmospheric CO2
    units:         ppmv
    grid_loc:      2110
    cell_methods:  time: mean xlon: mean ylat: mean

# Access SOCAT Data from Artemis

In [ ]:
#look at Val's pre SOCAT py file

path_SST = '/local/data/artemis/observations/NOAA_OI_SSTv2/processed/sst_1x1_mon_NOAAOIv2_198112-201908.nc'

path_SSS = '/local/data/artemis/observations/EN422_salinity/processed/sss_1x1_mon_EN422_197901-202012.nc'

path_xCO2 = #'https://gml.noaa.gov/ccgg/mbl/'
#make sure includes lat lon variations (via pressure field) and water vapor correction
# using pressure:
path_pres = '/local/data/artemis/observations/ERA5_MSLP/ERA5_MSLP_1x1_1979-2021.nc' #make sure in atmospheres
#expect 300 to 450 depending on year

path_chlA = '/local/data/artemis/observations/GlobColour/processed/globcolour-L3m-GSM_chl_1x1_monthly_198201-202112.nc'

path_MLD = '/local/data/artemis/observations/MLD_deBoyerMontegut/val_processed/mld_deboyer_1x1_clim_198201-201712.nc'
#LOOK AT Val's code, should be able to just add additional years, see if jan 2017 same as jan 2000
path_pCO2 = '/local/data/artemis/observations/SOCAT/processed/SOCATv2022_pCO2_gridded_monthly.nc'
#IF need to add CESM, /local/data/artemis/observations/CESM_hindcast/processed 

#ds1 = xr.open_dataset(‘/data/artemis/workspace/vbennington/GlobColour/originals/L3m_GLOB_CHL1_199801-202012.nc’)
#ds2 = xr.open_dataset(‘/data/artemis/workspace/afay/GlobColour/originals/L3m_GLOB_CHL1_202001-202112.nc’)

In [ ]:
df = ldeo_res_flux
lon = df.lon
print(lon.shape)
lon2 = np.where(lon < 0, lon+360, lon)
# # print(lon2)
df.coords['lon'] = (lon2)
df = df.sortby(df.lon)
# # print(df)
df[300,:,:].plot()
# # print(df.lon)
ldeo_res_flux360 = df
print(ldeo_res_flux360.dims)
print(ldeo_res_flux360.shape)
del df

In [ ]:
ds_pCO2 = xr.open_dataset(path_pCO2)
ds_SST = xr.open_dataset(path_SST)
ds_chlA = xr.open_dataset(path_chlA)

In [ ]:
print(ds_chlA) #NOTE WILL NEED TO STANDARDIZE LONGITUDE, and will need to standardize time
# check Val's scripts for time and lon conversions
# define a function for switching longitudes, see Amanda's example

In [ ]:
print(ds_pCO2)

In [ ]:
ds_pCO2.socat_mask.plot()

In [ ]:
dickson = pd.read_csv('/data/artemis/workspace/vbennington/Dickson_atmos/Dickson_monthly_pco2.csv')

In [ ]:
dickson.head() #only varies with time, not lat and lon

In [ ]:
# take time series of xCO2 and multiply by time and space-varying pressure